In [1]:
!pip install scikit-learn

# Step 2: Import necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer


In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.ensemble import RandomForestClassifier

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
news = pd.read_csv(r"C:\Users\91970\Downloads\bbc-text.csv")
news

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [8]:
news['category'].value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [9]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [10]:
corpus = []
for i in range(0, news.shape[0]):
    text = re.sub('[^a-zA-Z]', ' ', news['text'][i])#Removing Punctuation
    
    text = text.lower()
    text = text.split()
    
    lm = WordNetLemmatizer()
    
    text = [lm.lemmatize(word) for word in text if not word in stop_words]
    text = " ".join(text)
    corpus.append(text)
    

In [11]:
corpus[22]

'bates seal takeover ken bates completed takeover leeds united year old former chelsea chairman sealed deal gmt friday bought stake club said delighted stepping mantel fantastic club recognise leeds great club fallen hard time lot hard work ahead get club back belongs premiership help fan everything bates bought stake guise geneva based company known forward sport fund revealed part plan buy back leeds elland road stadium thorp arch training ground due course going tough jon first task stabilise cash flow sort remaining creditor bates added light end long tunnel past year matter firefighting start running club outgoing leeds chairman gerald krasner said deal ensures medium long term survival club believe mr bates proposal totally benefit club content mr bates leeds united continue consolidate move forward took leeds united march club debt since date board succeeded reducing debt worked tirelessly solve problem leeds united eighty percent problem already overcome came agreement mr bates

In [12]:
#Mostfrequently occuring words 
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted (words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]

In [13]:
#Convert most freq words to dataframe for plotting bar plot
top_words = get_top_n_words(corpus, n=20)
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]
top_df

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {"isn't", 'during', 'her', 'which', 'being', 'll', 'it', 'than', 'weren', "it's", 'myself', 'most', 'herself', 'his', 'yourselves', 'all', 'they', 'under', 'shouldn', 'how', 'into', "shan't", 'off', 'have', 'isn', 'a', 'haven', 'can', "should've", 'mustn', 'has', 'for', "hasn't", 'are', 'if', 't', "mustn't", 'own', 'hers', 'does', 'with', 'again', "needn't", 'wasn', 'the', 'theirs', 'or', 'y', 'here', 'further', 've', 'after', 'on', 'between', 'why', "didn't", 'each', 'doing', "won't", 'its', 'do', 'will', 'doesn', 'been', 'up', 'as', 'against', 'but', 'your', 'over', 'until', 'no', 'couldn', 'ain', 'now', "you're", 'same', "you'd", 'once', 'him', "wasn't", 'himself', 'aren', "couldn't", "shouldn't", 'of', 'from', 're', 'down', 'there', 'what', 'above', 'so', 'mightn', 'nor', 'an', 'such', 'very', 'themselves', 'this', 'only', 'shan', 'our', 'won', "d

In [14]:
news.columns

Index(['category', 'text'], dtype='object')

#Feature EXTRACTION

In [50]:
#Creating the badg of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = TfidfVectorizer(max_features=4000, ngram_range=(1,3))
x = cv.fit_transform(corpus).toarray()
y = news.drop(['text'], axis = 1)

In [52]:
print(x.shape, y.shape)

(2225, 4000) (2225, 1)


In [53]:
#Splitting into Training and Testing Dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.30, random_state=0)

In [54]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(1557, 4000) (1557, 1)
(668, 4000) (668, 1)


# Naive Bayes Classifier

In [56]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x_train, y_train)

GaussianNB()

In [57]:
y_pred = classifier.predict(x_test)


In [58]:
#Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[128,   1,  10,   3,  10],
       [  3,  90,   4,   0,  12],
       [  2,   0, 109,   1,   1],
       [  3,   2,   0, 170,   2],
       [  3,   2,   0,   0, 112]], dtype=int64)

In [59]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9116766467065869


# Linear Classifer

In [28]:
from sklearn.linear_model import LogisticRegression
classifier_LR = LogisticRegression(random_state = 0)
classifier_LR.fit(x_train,y_train)

LogisticRegression(random_state=0)

In [29]:
y_pred_LR = classifier_LR.predict(x_test)


In [31]:
cm_LR = confusion_matrix(y_test, y_pred)
cm_LR

array([[131,   1,  10,   2,   8],
       [  3,  91,   4,   0,  11],
       [  2,   0, 109,   1,   1],
       [  2,   1,   0, 172,   2],
       [  3,   1,   0,   0, 113]], dtype=int64)

In [32]:
print(accuracy_score(y_test, y_pred_LR))

0.9745508982035929


# CLASSIFICATION USING SVM

In [33]:
from sklearn.svm import SVC
classifier_SVM = SVC(kernel='linear')

classifier_SVM.fit(x_train,y_train)

SVC(kernel='linear')

In [34]:
y_pred_SVM = classifier_SVM.predict(x_test)

In [35]:
cm_LR = confusion_matrix(y_test, y_pred)
cm_LR

array([[131,   1,  10,   2,   8],
       [  3,  91,   4,   0,  11],
       [  2,   0, 109,   1,   1],
       [  2,   1,   0, 172,   2],
       [  3,   1,   0,   0, 113]], dtype=int64)

In [36]:
print(accuracy_score(y_test, y_pred_LR))

0.9745508982035929


# RANDOM FOREST


In [37]:
from sklearn.ensemble import RandomForestClassifier
classifier_RF = RandomForestClassifier(n_estimators = 500, random_state=123, verbose=4)
classifier_RF.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
b

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   11.5s finished


RandomForestClassifier(n_estimators=500, random_state=123, verbose=4)

In [38]:
y_pred_RF = classifier_RF.predict(x_test)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.3s finished


In [60]:
cm_RF = confusion_matrix(y_test, y_pred)
cm_RF

array([[128,   1,  10,   3,  10],
       [  3,  90,   4,   0,  12],
       [  2,   0, 109,   1,   1],
       [  3,   2,   0, 170,   2],
       [  3,   2,   0,   0, 112]], dtype=int64)

In [40]:
print(accuracy_score(y_test, y_pred_RF))

0.968562874251497


# FEATURE IMPORTANCE (to identify imp words from classifer)

In [61]:
#Convert to df
tf_vect_df = pd.DataFrame(x, columns = cv.get_feature_names_out())
tf_vect_df

,aaa,ability,able,abroad,absence,absolute,absolutely,abuse,abused,ac,...,young,young people,younger,youngster,yuan,yugansk,yuganskneftegas,yukos,zealand,zone
0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.041391,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.040477,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2221,0.0,0.000000,0.0,0.082443,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2222,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2223,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
feat_imp = classifier_RF.feature_importances_
feat_imp

array([9.53621570e-06, 1.11961392e-04, 3.53100353e-04, ...,
       1.77947138e-04, 5.02568816e-05, 4.76468355e-05])

In [70]:
feature_cols = count_vect_df.columns
feature_cols

Index(['aaa', 'ability', 'able', 'abroad', 'absence', 'absolute', 'absolutely',
       'abuse', 'abused', 'ac',
       ...
       'young', 'young people', 'younger', 'youngster', 'yuan', 'yugansk',
       'yuganskneftegas', 'yukos', 'zealand', 'zone'],
      dtype='object', length=4000)

In [71]:
feat_imp_dict = dict(zip(feature_cols, classifier_RF.feature_importances_))
feat_imp = pd.DataFrame.from_dict(feat_imp_dict, orient= 'index')
feat_imp.sort_values(by=0, ascending= False).head(20)

,0
bn,0.014514
film,0.014005
game,0.012087
market,0.011839
firm,0.010848
minister,0.010437
technology,0.010332
user,0.010128
government,0.010075
star,0.009804


In [64]:
from sklearn.svm import SVC
classifer_SVM = SVC(kernel = "linear")
classifier_SVM.fit(x_train, y_train)

SVC(kernel='linear')

In [67]:
y_pred_SVM = classifier_SVM.predict(x_test)

In [68]:
print(accuracy_score(y_test, y_pred_SVM))

0.9775449101796407
